In [1]:
import numpy as np
import tensorflow as tf

Reading the data from text files to  numpy arrays
- affirmative      - 0
- conditional      - 1
- doubt_question   - 2
- emphasis         - 3
- negative         - 4
- relative         - 5
- topics           - 6
- wh_question      - 7
- yn_question      - 8

In [2]:
files_list=['affirmative','conditional','doubt_question','emphasis','negative','relative','topics','wh_question','yn_question']

def read_data(index):
    '''Takes index indexing the files_list list and reads the data from corresponding text files and returns it
        Also removes first row and first column from the data as part of first pre processing step
        Concatenates data from two persons a and b'''
    index=int(index)
    x_a = np.loadtxt('grammatical_facial_expression/a_'+files_list[index]+'_datapoints.txt', skiprows=1)
    x_b = np.loadtxt('grammatical_facial_expression/b_'+files_list[index]+'_datapoints.txt', skiprows=1)
    y_a = np.loadtxt('grammatical_facial_expression/a_'+files_list[index]+'_targets.txt')
    y_b = np.loadtxt('grammatical_facial_expression/b_'+files_list[index]+'_targets.txt')
    x = np.concatenate((x_a, x_b))
    y = np.concatenate((y_a, y_b))
    return x[:,1:],y 

def read_data_one(index, name):
    '''Takes index indexing the files_list list and reads the data from corresponding text files and returns it
        Also removes first row and first column from the data as part of first pre processing step
        Reads data only from one person name taken as argument a/b '''
    index=int(index)
    x_ = np.loadtxt('grammatical_facial_expression/'+name+'_'+files_list[index]+'_datapoints.txt', skiprows=1)
    y_ = np.loadtxt('grammatical_facial_expression/'+name+'_'+files_list[index]+'_targets.txt')
    return x_[:,1:],y_



### Data Preprocessing

- Normalization
- Making both the classes of equal size for handling class imbalance problem
- Shuffling the data to avoid any further problems
- Converting y-labels as one hot

In [3]:
def normalize_data(x):
    ''' Normalizes all features of the data
        Second step of data preprocessing'''
    size=np.shape(x)[1]
    mean=np.mean(x,axis=0)
    sigma=np.std(x,axis=0)
    norm_x = (x-mean)/sigma
    return norm_x

def split_data(x, y, ratio):
    '''Splits the dataset according to the ratio given.
        Shuffles the data using random shuffling before splitting
        Converts the y labels as one hot vectors'''
    y=y[:,np.newaxis]
    X=np.concatenate((x,y),axis=1)
    np.random.shuffle(X)
    train_size = int(np.shape(X)[0] * ratio)
    y_train = np.zeros((train_size,2))
    y_test = np.zeros((X.shape[0]-train_size, 2))
    y1=X[:train_size,-1]
    y2=X[train_size:,-1]
    y_train[y1==0,0]=1
    y_train[y1==1,1]=1
    y_test[y2==0,0]=1
    y_test[y2==1,1]=1
    return X[:train_size,:-1], y_train, X[train_size:,:-1 ], y_test

In [4]:
x_data, y_data = read_data_one(0, 'a') #read data 5->relative #person A
x_data = normalize_data(x_data) #normalize
#np.savetxt('a_relative.csv',x_data,delimiter=',')
x_data = x_data*100
x_train, y_train, x_test, y_test = split_data(x_data, y_data, 0.8) #test-train split

### Defining required placeholders and Variables

In [5]:
x_size = np.shape(x_train)[1]
# Defining Placeholders for input data and labels
X = tf.placeholder(tf.float32, (None, x_size), name='X')
Y = tf.placeholder(tf.float32, (None, 2), name='Y')

In [6]:
# Defining variables for weights of NN
w1 = tf.Variable(tf.random_normal((300,1)), name="w1") # Weights for hidden layer 1
b1 = tf.Variable(tf.reshape(tf.random_normal((100,1)),[100]), name="b1")     # Bias for hidden layer 1

w2 = tf.Variable(tf.random_normal((100,1)), name='w2') # Weights for hidden layer 2
b2=tf.Variable(tf.reshape(tf.random_normal((10,1)),[10]), name='b2')        # Bias for hidden layer 2

w3 = tf.Variable(tf.random_normal((10,2)),name='w3')  # Weights for output layer
b3 = tf.Variable(tf.random_normal((1,2)), name='b3')  # Bias for output layer

### Defining nodes connections

In [7]:
# First Layer

i=0
z_i = tf.nn.relu(tf.add(tf.matmul(X[:,i:i+3], w1[i:i+3,:]), b1[int(i/3)]), name='layer_1') #first node of hidden layer 1
first_layer = z_i

for i in range(3,300,3):
    z_i = tf.nn.relu(tf.add(tf.matmul(X[:,i:i+3], w1[i:i+3,:]), b1[int(i/3)])) #Remaining nodes of hidden layer 2
    first_layer = tf.concat((first_layer, z_i), axis=1)
#print(w1)
#print(first_layer)

In [8]:
# Second Layer

#Left Eye
z21 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,0:8], w2[0:8,:]), b2[0]), name='layer_2')
second_layer = z21

#Right Eye
z22 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,8:16], w2[8:16,:]), b2[1]))
second_layer = tf.concat((second_layer, z22), axis=1)

#Left Eyebrow
z23 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,16:26], w2[16:26,:]), b2[2]))
second_layer = tf.concat((second_layer, z23), axis=1)

#Right Eyebrow
z24 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,26:36], w2[26:36,:]), b2[3]))
second_layer = tf.concat((second_layer, z24), axis=1)

#Nose
z25 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,36:48], w2[36:48,:]), b2[4]))
second_layer = tf.concat((second_layer, z25), axis=1)

#Mouth
z26 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,48:68], w2[48:68,:]), b2[5]))
second_layer = tf.concat((second_layer, z26), axis=1)

#Face Contour
z27 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,68:87], w2[68:87,:]), b2[6]))
second_layer = tf.concat((second_layer, z27), axis=1)

# Left, right iris + nose tip
z28 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,87:90], w2[87:90,:]), b2[7]))
second_layer = tf.concat((second_layer, z28), axis=1)

#Line above left eyebrow
z29 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,90:95], w2[90:95,:]), b2[8]))
second_layer = tf.concat((second_layer, z29), axis=1)

#Left Eye
z210 = tf.nn.relu(tf.add(tf.matmul(first_layer[:,95:], w2[95:,:]), b2[9]))
second_layer = tf.concat((second_layer, z210), axis=1)

#print(w2)
#print(second_layer)

In [9]:
#Output layer

z3 = tf.add(tf.matmul(second_layer, w3), b3)
out_weights = tf.reshape(w3, [1,20])

output=tf.nn.softmax(z3, name='output')

### Accuracy, Cost and Training step tensors

In [10]:
#Accuracy
accuracy = tf.reduce_mean(tf.cast( tf.equal(tf.argmax(Y,1), tf.argmax(output,1)),tf.float32), name='accuracy')

In [11]:
#Cost function

cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output), name='cost')
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

### Training

In [12]:
sess =  tf.Session() 
writer = tf.summary.FileWriter('./project_graph', sess.graph)
init = tf.global_variables_initializer()
sess.run(init)
checkpoint=10
for e in range(1300):
    _,l = sess.run([train_step, cost], feed_dict={X:x_train, Y:y_train})
    if(e==checkpoint):
        acc = sess.run(accuracy, feed_dict={X:x_train, Y:y_train})
        print('epoch: '+str(e)+' accuracy: ' +str(acc) + ' loss: '+str(l))
        checkpoint=checkpoint*2
writer.close()

epoch: 10 accuracy: 0.803298 loss: 437.1862
epoch: 20 accuracy: 0.8351001 loss: 406.20282
epoch: 40 accuracy: 0.8515901 loss: 391.93997
epoch: 80 accuracy: 0.85276794 loss: 390.96027
epoch: 160 accuracy: 0.85276794 loss: 390.96
epoch: 320 accuracy: 0.85276794 loss: 390.95984
epoch: 640 accuracy: 0.85276794 loss: 390.95975
epoch: 1280 accuracy: 0.85276794 loss: 390.95966


### Testing

In [13]:
acc_test = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
print('Test Accuracy: '+ str(acc_test))

Test Accuracy: 0.8309859
